In [1]:
import re
import time
import json
import csv
import unicodedata
import openai
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm
from pprint import pprint
from file_save import *

In [2]:
def compute_accuracy(tuples):
    """Measure the GPT3.5 model's accuracy."""
    n_correct = int(0)
    n_correct_substr = int(0)
    n_tuples = len(tuples)
    for t in tuples:
        gpt_choices = t["bloom_choices"]
        for item in range(len(gpt_choices)):
            gpt_choices[item] = gpt_choices[item].strip()
            
        obj_labels = t["obj_label"]

        # An answer is correct if it is one of candidate answers
        n_correct += int(any([c in obj_labels for c in gpt_choices]))

        # An answer is correct if any of the candidate answers is a substr of the answer
        n_correct_substr += int(any([o in c for o in obj_labels for c in gpt_choices]))

    return {
        "Total number of tuples": n_tuples,
        "# of correct answers (Exact match)": n_correct,
        "% of correct answers (Exact match)": round(100 * n_correct / n_tuples, 1),
        "# of correct answers (Overlap)": n_correct_substr,
        "% of correct answers (Overlap)": round(100 * n_correct_substr / n_tuples, 1),
    }

"""
def is_word_in_string(word, string):
    pattern = r'\b' + re.escape(word) + r'\b'
    return bool(re.search(pattern, string))
"""

def is_string_in(text, phrase):
    return f' {text.lower()} ' in f' {phrase.lower()} '

def lowercase_first_letter(input_string):
    if len(input_string) > 0:
        return input_string[0].lower() + input_string[1:]
    return input_string  # Return input as is for an empty string

## Find the same obj

In [8]:
path1 = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/dataset/dlama-v1/test/en-extend/"
path2 = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/dataset/dlama-v1/test/en-es/"
path3 = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/dataset/dlama-v1/test/en-ar/"

file_paths1 = glob.glob(path1 + '*')
file_paths2 = glob.glob(path2 + '*')
file_paths3 = glob.glob(path3 + '*')

In [21]:
subs1_all = []
subs2_all = []
subs3_all = []
for i in range(len(file_paths1)):
    predicate = file_paths1[i].split("\\")[-1]
    subs1 = [a["sub_label"] for a in load_jsonl_file(file_paths1[i])]
    subs1_all.extend(subs1)
    subs2 = [a["sub_label"] for a in load_jsonl_file(file_paths2[i])]
    subs2_all.extend(subs2)
    subs3 = [a["sub_label"] for a in load_jsonl_file(file_paths3[i])]
    subs3_all.extend(subs3)
    
    common_elements = set(subs1) & set(subs2) 
    num_common_elements = len(common_elements)
    print(len(subs1))
    print(len(subs2))
    print(len(subs3))
    print("for Predicate: "+predicate+ "， it has " + str(num_common_elements) + " same subjects")
    
print(len(subs1_all))
print(len(subs2_all))
print(len(subs3_all))
common_elements = set(subs1_all) & set(subs2_all) 
num_common_elements = len(common_elements)
print("for Predicate: "+predicate+ "， it has " + str(num_common_elements) + " same subjects")
    

1000
1000
1000
for Predicate: P103_general_WESTERN_COUNTRIES.jsonl， it has 840 same subjects
1000
1000
1000
for Predicate: P106_general_WESTERN_COUNTRIES.jsonl， it has 555 same subjects
1000
1000
1000
for Predicate: P1303_general_WESTERN_COUNTRIES.jsonl， it has 749 same subjects
1000
1000
1000
for Predicate: P136_general_WESTERN_COUNTRIES.jsonl， it has 783 same subjects
26
26
26
for Predicate: P1376_general_WESTERN_COUNTRIES.jsonl， it has 26 same subjects
1000
1000
1000
for Predicate: P1412_general_WESTERN_COUNTRIES.jsonl， it has 512 same subjects
1000
1000
1000
for Predicate: P17_general_WESTERN_COUNTRIES.jsonl， it has 605 same subjects
467
465
468
for Predicate: P190_general_WESTERN_COUNTRIES.jsonl， it has 454 same subjects
1000
1000
1000
for Predicate: P19_general_WESTERN_COUNTRIES.jsonl， it has 304 same subjects
1000
1000
1000
for Predicate: P20_general_WESTERN_COUNTRIES.jsonl， it has 258 same subjects
1000
1000
1000
for Predicate: P264_general_WESTERN_COUNTRIES.jsonl， it has 673 s

# find the wrong obj

In [14]:
#gpt
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt4-result//result/en-extend/_P190_general_WESTERN_COUNTRIES_output.jsonl"
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt4-result//result/en-extend/_P1376_general_Asia_output.jsonl"

#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt4-result/result/en-ar/_P17_general_WESTERN_COUNTRIES_output.jsonl"
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt3.5-result/ar/_P449_general_ARAB_REGION_output.jsonl"

#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt4-result/result/en-es/_P17_general_WESTERN_COUNTRIES_output.jsonl"
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt4-result/result/en-es/_P20_general_SOUTH_AMERICA_output.jsonl"

path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt3.5-result/result/en/_P1303_general_WESTERN_COUNTRIES_output.jsonl"
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt3.5-result/result/en/_P1303_general_Asia_output.jsonl"

#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt4-result/result/zh-cn-trans-extend/_P449_general_WESTERN_COUNTRIES_output.jsonl"
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt4-result/result/zh-cn-trans-extend/_P449_general_Asia_output.jsonl"


In [4]:
#LLama
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/llama2-7b-chat/result-Llama2/en-ar/llama-2-7b-chat_P17_general_WESTERN_COUNTRIES_output.jsonl"
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/llama2-7b-chat/result-Llama2/en-ar/llama-2-7b-chat_P103_general_ARAB_REGION_output.jsonl"



In [68]:
# Bloom
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/bloom/bloom-3b/en-es/bloom-3b_P264_general_WESTERN_COUNTRIES_output.jsonl"
#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/bloom/bloom-1b1/en-ar/bloom-1b1_P136_general_ARAB_REGION_output.jsonl"

#path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result//bloom/bloom-1b1/en-es/bloom-1b1_P30_general_SOUTH_AMERICA_output.jsonl"


In [16]:
def compute_accuracy(tuples):
    """Measure the model's accuracy."""
    n_correct = int(0)
    n_correct_substr = int(0)
    n_tuples = len(tuples)
    for t in tuples:
        #gpt_choices = t["bloom_choices"]
        gpt_choices = t["gpt3.5-turbo_choices"]
        for item in range(len(gpt_choices)):
            gpt_choices[item] = gpt_choices[item].strip()
            
        obj_labels = t["obj_label"]

        # An answer is correct if it is one of candidate answers
        n_correct += int(any([c in obj_labels for c in gpt_choices]))

        # An answer is correct if any of the candidate answers is a substr of the answer
        count = 0
        
        for c in gpt_choices:
            for o in obj_labels:
                if not o.isupper():
                    o = o.lower()
                    if o in c.lower():
                        count = 1
                        break
                else:
                    if o in c:
                        count = 1
                        break
            if count == 1:
                break
        
        n_correct_substr += count         
        #n_correct_substr += int(any([o.lower() in c.lower() for o in obj_labels for c in gpt_choices]))
        #n_correct_substr += int(any([is_string_in(o.lower(), c.lower().replace(".", " ").replace(",", " ")) for o in obj_labels for c in gpt_choices]))
    return {
        "Total number of tuples": n_tuples,
        "# of correct answers (Exact match)": n_correct,
        "% of correct answers (Exact match)": round(100 * n_correct / n_tuples, 1),
        "# of correct answers (Overlap)": n_correct_substr,
        "% of correct answers (Overlap)": round(100 * n_correct_substr / n_tuples, 1),
    }


In [18]:


tuples = load_jsonl_file(path)
print(compute_accuracy(tuples))
false_tuples_original = []
false_tuples_captial_consider = []
false_tuples_low = []
false_tuples_occo = []

n_correct = 0
n_correct_substr_occo = 0
n_correct_substr = 0

n_tuples = len(tuples)
for t in tuples:
    gpt_choices = t["gpt3.5-turbo_choices"]
    #gpt_choices = t["Llama2_choices"]
    #gpt_choices = t["gpt4_choices"]
    #gpt_choices = t["bloom_choices"]
    for item in range(len(gpt_choices)):
        gpt_choices[item] = gpt_choices[item].strip()

    obj_labels = t["obj_label"]

    # An answer is correct if it is one of candidate answers
    is_correct_origin = False
    is_correct = False
    is_correct1 = False
    is_correct_occo = False
    for c in gpt_choices:
        for o in obj_labels:
            c_copy = c
            
            if o in c:
                is_correct_origin = True
        
            if o.lower() in c.lower():
                is_correct1 = True
                
            if not o.isupper():
                if o.lower() in c.lower():
                    is_correct = True
                    
            else:
                if o in c:
                    is_correct = True
                
                
    if is_correct1 == False:
        false_tuples_low.append(t)
        
    if is_correct == False:
        false_tuples_captial_consider.append(t)
        
    if is_correct_origin == False:
        false_tuples_original.append(t)
        
    n_correct += int(any([c in obj_labels for c in gpt_choices]))

    # An answer is correct if any of the candidate answers is a substr of the answer
    n_correct_substr += int(any([o in c for o in obj_labels for c in gpt_choices]))
    count = 0
    for c in gpt_choices:
            for o in obj_labels:
                if o.lower() in c.lower() or c.lower() in o.lower():
                    is_correct_occo = True
                    count = 1
                    break
    if count == 0:
        false_tuples_occo.append(t)
                
    n_correct_substr_occo += count   
    
print("Current is <", path.split("/")[-2], "> ", path.split("/")[-1].replace("_output.jsonl", ""))
    
print(n_tuples)

print("lower but captial consider false:",len(false_tuples_captial_consider))
print("lower applied false:", len(false_tuples_low))
print("occo false:", len(false_tuples_occo))

print("--------------------------------------------------")

print("correct:")
print("original:",n_correct_substr)
print("lower but captial consider:", n_tuples - len(false_tuples_captial_consider)) # original answer eg. o in c
print("lower:", n_tuples - len(false_tuples_low)) #lower answer, ge. o.lower in c.lower
print("occo: ", n_correct_substr_occo) #occo ,eg. o.lower() in c.lower() or c.lower() in o.lower()

print("-------------------------------------------------")
print("P@1")
print("original:", (n_tuples - len(false_tuples_original))/n_tuples) # original answer eg. o in c
print("lower but captial consider:", (n_tuples - len(false_tuples_captial_consider))/n_tuples) # original answer eg. o in c
print("lower:", (n_tuples - len(false_tuples_low))/n_tuples) #lower answer, ge. o.lower in c.lower
print("occo: ", n_correct_substr_occo/n_tuples) #occo ,eg. o.lower() in c.lower() or c.lower() in o.lower()

{'Total number of tuples': 1000, '# of correct answers (Exact match)': 319, '% of correct answers (Exact match)': 31.9, '# of correct answers (Overlap)': 542, '% of correct answers (Overlap)': 54.2}
Current is < en >  _P1303_general_WESTERN_COUNTRIES
1000
lower but captial consider false: 458
lower applied false: 458
occo false: 430
--------------------------------------------------
correct:
original: 332
lower but captial consider: 542
lower: 542
occo:  570
-------------------------------------------------
P@1
original: 0.332
lower but captial consider: 0.542
lower: 0.542
occo:  0.57


## check the false

In [38]:
for a in false_tuples_captial_consider[:5]:
    print(a["sub_label"])
    print(a["obj_label"])
    print("\n")
    print(a["bloom_choices"])
    print("\n")
    print("-----------------------------------------------")
    

Ali Mohammed al-Hiq
['🇱🇧', 'Republic of Lebanon', 'Lebanon', 'LBN', 'lb', 'Lebanese Republic', 'Lubnan']


['Algeria']


-----------------------------------------------
Soad Ibrahim
['Arab Republic of Egypt', 'eg', 'EGY', 'Arab Rep. Egypt', 'Republic of Egypt', '🇪🇬', 'Rep. Egypt', 'Egypt']


['Malaysia']


-----------------------------------------------
Mohammed bin Rashid Al Maktoum
['UAE', 'Emirates', 'U.A.E.', 'ae', 'the United Arab Emirates', 'the U.A.E.', 'the UAE', '🇦🇪', 'the Emirates', 'United Arab Emirates']


['Dubai']


-----------------------------------------------
Hakim Ziyech
['Maroc', 'the Netherlands', 'MAR', 'Netherlands', 'al-Maġrib', 'Lmaġrib', '🇳🇱', 'Kingdom of Morocco', 'Marocco', 'Morocco', 'Kingdom of the Netherlands', 'ma', 'NL', '🇲🇦']


['Algeria']


-----------------------------------------------
Khalid Abdulrahman
['Saudi Arabia', 'sa', 'Kingdom of Saudi Arabia', '🇸🇦', 'KSA']


['Malaysia']


-----------------------------------------------


## check the differences

In [49]:
#P449
c = false_tuples_original
for a in false_tuples_captial_consider: #大
    for b in false_tuples_original: #小
        if a["sub_label"] == b["sub_label"]:
            c.remove(b)
len(c)            
            

28

In [769]:
#P449
c = false_tuples_captial_consider
for a in false_tuples_low: #大
    for b in false_tuples_captial_consider: #小
        if a["sub_label"] == b["sub_label"]:
            c.remove(b)
len(c)            
            

8

In [758]:
c = false_tuples_captial_consider
for a in false_tuples_occo: #大
    for b in false_tuples_captial_consider: #小
        if a["sub_label"] == b["sub_label"]:
            c.remove(b)
len(c)            
            

17

In [50]:
count = 0
for c1 in c:
    is_in = False
    for cc in c1["obj_label"]:
        if cc.lower() == c1["bloom_choices"][0]:
        #if cc.lower() == c1["Llama2_choices"][0]:
            is_in = True
            count += 1
            break
    if not is_in:
        print(c1["sub_label"])
        print(c1["obj_label"])
        print("\n")
        print(c1["bloom_choices"])
        print("\n")
        print("-----------------------------------------------")
print(count)


Rafael Nadal
['athletes', 'model', 'sportswoman', 'sportsman', 'poser', 'fashion model', 'creator', 'sportsperson', 'author', 'athlete', 'tennis player', 'sitter']


['Tennis player']


-----------------------------------------------
Ricky Martin
['theatre actress', 'creator', 'scenic artist', 'actors', 'theatrical actress', 'penman', 'contributor', 'TV actor', 'television actor', 'theater actor', 'TV actress', 'musician', 'vocalist', 'songstress', 'performer', 'artist', 'theater actress', 'musical artist', 'singer', '🧑\u200d🎤', 'television actress', 'theatre actor', 'author', 'sitter', 'actress', 'model', 'stage actor', 'stage actress', 'fashion model', 'composer', 'Performers', 'authors', 'poser', 'actresses', 'theatrical actor', 'actor', 'penwoman', 'music hall performer', 'performing artist', 'writers', 'writer', 'musicians, singers and composers']


['Musician']


-----------------------------------------------
Charles Le Brun
['architect', 'designer', 'author of the drawing', 'pr

In [51]:
count = 0
for item in c:
    for item1 in item["obj_label"]:
        if item1.lower() in item["bloom_choices"][0].lower():
            print("obj: ", item1)
            print("model: ", item["bloom_choices"][0])
            print("\n")
            count += 1
print(count)

obj:  tennis player
model:  Tennis player


obj:  musician
model:  Musician


obj:  architect
model:  Architect


obj:  musician
model:  Musician


obj:  director
model:  Director of Animation


obj:  actor
model:  Actor


obj:  racing driver
model:  Racing driver


obj:  musician
model:  Musician


obj:  actress
model:  Actress


obj:  director
model:  Director of photography


obj:  football manager
model:  Football Manager


obj:  manager
model:  Football Manager


obj:  musician
model:  Musician


obj:  director
model:  Director of photography


obj:  actor
model:  Actor


obj:  queen
model:  Queen of England


obj:  painter
model:  Painter


obj:  musician
model:  Musician


obj:  musician
model:  Musician


obj:  director
model:  Director of photography


obj:  director of photography
model:  Director of photography


obj:  musician
model:  Musician


obj:  director
model:  Director of Photography


obj:  theologian
model:  Theologian


obj:  footballer
model:  French Footballer


In [35]:
print(len(false_tuples_original))
for i in tuples:
    if i in false_tuples_original:
        tuples.remove(i)

count = 0
print(len(tuples))
for item in tuples[:10]:
    count += 1
    for item1 in item["obj_label"]:
        if item1.lower() in item["bloom_choices"][0].lower():
            print(count)
            print("obj: ", item1)
            print("model: ", item["bloom_choices"][0])
            print("\n")

2
466
10
obj:  Valencia
model:  Valencia is the sister city




In [611]:

for a in false_tuples_origin[10:20]:
    print(a["sub_label"])
    print(a["obj_label"])
    print("\n")
    print(a["gpt3.5-turbo_choices"])
    print("\n")
    print("-----------------------------------------------")

Shaabiat Al Cartoon
['Sama Dubai']


['MBC']


-----------------------------------------------
Nsibti Laaziza
['Nessma TV']


['Tunisian National Television.']


-----------------------------------------------
L'Maktoub
['2M TV']


['MTV']


-----------------------------------------------
El Wa3ra
['Echourouk TV', 'Echorouk TV']


['El Wa3ra originally aired on Saudi Broadcasting Authority.']


-----------------------------------------------
El Khawa
['El Djazairia One']


['Al Jadeed']


-----------------------------------------------
Samarqand
['TeN TV', 'Abu Dhabi drama tv series Bulgaria 2014 2016 - 2023', 'AD Drama', 'Rotana Gulf', 'Abu Dhabi Drama', 'Future Television', 'Al Tahrir TV', 'Al Tahrir', 'Mbc 1', 'MBC1 mbc drama', 'Abu Dhabi TV', 'Future TV', 'MBC 1', 'Drama TV']


['Uzbekistan National Television']


-----------------------------------------------
The End
['ON E', 'ONTV', 'ON TV', 'ON']


['HBO']


-----------------------------------------------
Naouret El Hawa
['Télé

In [326]:

tuples = load_jsonl_file(path.replace("\\", "/"))
false_tuples3 = []
false_tuples4 = []

n_correct = 0
n_correct_substr = 0
n_tuples = len(tuples)
for t in tuples:
    gpt_choices = t["gpt4_choices"]
    for item in range(len(gpt_choices)):
        gpt_choices[item] = gpt_choices[item].strip()

    obj_labels = t["obj_label"]

    # An answer is correct if it is one of candidate answers
    is_correct = False
    is_correct1 = False
    for c in gpt_choices:
        for o in obj_labels:
            if is_word_in_string(o.lower(),c.lower()):
                is_correct = True
            if is_word_in_string(o,c):
                is_correct1 = True
                
    if is_correct1 == False:
        false_tuples4.append(t)
        
    if is_correct == False:
        false_tuples3.append(t)
        
    n_correct += int(any([c in obj_labels for c in gpt_choices]))

    # An answer is correct if any of the candidate answers is a substr of the answer
    n_correct_substr += int(any([o in c for o in obj_labels for c in gpt_choices]))
    
print(n_tuples)
print(len(false_tuples3))
print(len(false_tuples4))
print("correct:")
print(n_tuples - len(false_tuples3))
print(n_correct_substr)

13
1
13
correct:
12
3


In [75]:
def is_word_in_string(word, string):
    pattern = r'\b' + re.escape(word) + r'\b'
    return bool(re.search(pattern, string))

word_to_check = "New"
string_to_search = "I have a new city and a newer car."

is_word_in_string(word_to_check, string_to_search)

False

In [32]:
false_tuples3 = false_tuples #ar

In [34]:
false_tuples2 = false_tuples # en

In [36]:
false_tuples1 = false_tuples #es

In [42]:
same_tuples = []
count = 0
for t in false_tuples2:
    for t2 in false_tuples1:
        
        
        if t['sub_label'] == t2['sub_label']:
            print(t)
            print(t2)
            print("\n")
            same_tuples.append(t)
            count += 1

print(count)

            

{'sub_label': 'Charles III', 'obj_label': ['United Kingdom of Great Britain and Northern Ireland', 'GB', 'The United Kingdom of Great Britain and Northern Ireland', 'G. B.', 'U.K.', 'UK', 'United Kingdom', 'G B', 'Great Britain', 'GBR', 'Great Britain and Northern Ireland', '🇬🇧', 'Britain', 'G B R', 'The UK', 'U. K.', 'U K', 'G.B.', 'G.B.R.'], 'gpt4_choices': ['monaco']}
{'sub_label': 'Charles III', 'obj_label': ['G B R', 'U. K.', 'GBR', 'United Kingdom', 'Britain', 'G B', 'United Kingdom of Great Britain and Northern Ireland', 'UK', 'Great Britain and Northern Ireland', 'G.B.R.', 'U K', 'G.B.', 'The UK', '🇬🇧', 'The United Kingdom of Great Britain and Northern Ireland', 'U.K.', 'G. B.', 'GB', 'Great Britain'], 'gpt4_choices': ['monaco']}


{'sub_label': 'Frédéric Chopin', 'obj_label': ['Tsarist Russia', 'Moscovy', 'Moscovy or Russia', 'Empire of Russia', 'Congress Poland', 'Kingdom of Poland', 'Second Republic of France', 'Duchy of Warsaw', 'Rusia', 'Russian Poland', 'Tsardom of Poland

In [40]:
print(len(false_tuples1)) #
print(len(false_tuples2)) # 1-2: 9
print(len(false_tuples3)) #1-3: 34  2-3: 10

231
15
56


## Update obj_label

In [3]:
need_change_folder_path = 'C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/BLOOM/result-bloom/zh-cn-trans-extend/'

original_folder_path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/dataset/dlama-v1/asia-west/dlama/zh-cn-trans/"

PRETICATES = ["P1412", "P1376", "P1303", "P530", "P495", "P449", "P364", "P264", "P190", "P136", "P106", "P103", "P47", "P37", "P36", "P30", "P27", "P20", "P19", "P17"]

save_folder_path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/BLOOM/result-bloom/修复的result/"

need_change_file_paths = glob.glob(need_change_folder_path + '*')
need_change_file_paths[0].split("\\")[-1]

'bloom-1b1_P103_general_ASIA_output.jsonl'

In [11]:
for file in need_change_file_paths:
    #for bloom
    model_name = file.split("\\")[-1].split("_")[0]
    #######
    need_change_triples = load_jsonl_file(file)
    
    current_file_name = file.split("\\")[-1]
   
    output_path = save_folder_path + "zh-cn-trans-extend/" + current_file_name
    only_change_output_path = save_folder_path + "change/" + current_file_name
    
    current_file_name = current_file_name.replace(model_name+"_", "").replace("_output", "")
    
    original_triples = load_jsonl_file(original_folder_path + current_file_name)
    
    output_triples = []
    only_changed_triples = []
    
    for triple in need_change_triples:
        if len(triple["obj_label"]) == 0:
            for ori_triple in original_triples:
                if ori_triple["sub_label"] == triple["sub_label"]:
                    triple["obj_label"] = ori_triple["obj_label"]
                    triple["Modify_obj?"] = "Yes"
                    only_changed_triples.append(triple)
                    
        output_triples.append(triple)
    
    save_jsonl_file_no_EN(output_path, output_triples)    
    save_jsonl_file_no_EN(only_change_output_path, only_changed_triples)    
            
            
        
    
        

## Calculate Accuracy

In [7]:
save_folder_path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/BLOOM/result-bloom/修复的result/zh-cn-trans-extend/"

file_paths = glob.glob(save_folder_path + '*')


df_info = {}

lang = ["zh-cn-trans-extend"]
model_name_list = []
file_name_list = []
total_list = []
for i in lang:
    info = {}
    info["lang_list"] = []
    info["N_c_e"] = []
    info["P_c_e"] = []
    info["N_c_o"] = []
    info["P_c_o"] = []
    info["time_cost"] = []

    df_info[i] = info
# Iterate over the file paths and read each file
count = 0
columns = ['model name','file name','tuples numbers']
data = []
for file_path in file_paths:
    current_lang = "zh-cn-trans-extend"
    
    #for bloom
    model_name = file_path.split("\\")[-1].split("_")[0]
    model_name_list.append(model_name)
    #######
    
    start = time.perf_counter()
    output_triple = load_jsonl_file(file_path)
    current_file_name = file_path.split("\\")[-1]

    current_file_name= current_file_name.replace(".jsonl", "")
    current_result_list = list(compute_accuracy(output_triple).values())

    file_name_list.append(current_file_name)
    total_list.append(current_result_list[0])

    end = time.perf_counter()
    elapsed = end - start
    df_info[current_lang]["lang_list"].append(current_lang)
    df_info[current_lang]["N_c_e"].append(current_result_list[1])
    df_info[current_lang]["P_c_e"].append(current_result_list[2])
    df_info[current_lang]["N_c_o"].append(current_result_list[3])
    df_info[current_lang]["P_c_o"].append(current_result_list[4])
    df_info[current_lang]["time_cost"].append(elapsed)

    columns = ['model name','file name','tuples numbers']
    data = [model_name_list,file_name_list, total_list]
        
for ii in lang:
    columns.extend(['language','e', 'e%', 'o', 'o%', "time cost"])
    data.append(df_info[ii]["lang_list"])
    data.append(df_info[ii]["N_c_e"])
    data.append(df_info[ii]["P_c_e"])
    data.append(df_info[ii]["N_c_o"])
    data.append(df_info[ii]["P_c_o"])
    data.append(df_info[ii]["time_cost"])
data = tuple(data)

df = pd.DataFrame(data,index=columns).T
df.to_excel("C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/BLOOM/result-bloom/修复的result/test.xlsx")
        
    
    

In [26]:
load_jsonl_file(input_file_path)

[{'sub_uri': 'Q39',
  'obj_uri': ['Q1016',
   'Q1246',
   'Q142',
   'Q145',
   'Q148',
   'Q159',
   'Q159583',
   'Q16',
   'Q183',
   'Q184',
   'Q217',
   'Q218',
   'Q227',
   'Q252',
   'Q258',
   'Q28',
   'Q30',
   'Q347',
   'Q35',
   'Q38',
   'Q399',
   'Q40',
   'Q403',
   'Q408',
   'Q41',
   'Q414',
   'Q423',
   'Q43',
   'Q458',
   'Q77',
   'Q79',
   'Q794',
   'Q801',
   'Q833',
   'Q843',
   'Q854',
   'Q865',
   'Q884',
   'Q928',
   'Q96'],
  'sub_label': 'Switzerland',
  'obj_label': ['ID',
   'Taiwan',
   'Chunghwa Minkwo',
   'State of Libya',
   'el',
   'Egypt',
   'United States of America',
   'France',
   'European Union',
   'Bielaruś',
   'md',
   'Republic of China (Taiwan)',
   'Federation of Malaysia',
   '🇱🇾',
   'az',
   'Hellas',
   'Belarus',
   'Chair of Peter',
   'the US of America',
   'Taprobane',
   'Europe',
   'Korea (South)',
   'N. Korea',
   'eu',
   'Azərbaycan Respublikası',
   'Rep. Egypt',
   'id',
   '🇱🇮',
   'U. S. A.',
   'US of M

In [ ]:
input_file_path = "dlama-v1\en-es\P449_general_WESTERN_COUNTRIES_output.jsonl"
tuples = load_jsonl_file(input_file_path)
print(compute_accuracy(tuples))